# Visualization of Transformer Prediction on a map

In [1]:
import torch
import torch.nn as nn
from tptrans import TPTrans
from dataloader import load_parquet, preprocess_data, SlidingWindowDataset
from plotting import plot_testresult_sample
from torch.utils.data import DataLoader

In [ ]:
#in your case it will be
#dataset = "dataset"
dataset = "aisdk-2025-02-27"

In [ ]:
df = load_parquet(dataset, k=5)
df = preprocess_data(df)

dataset = SlidingWindowDataset(
    df,
    max_diff_per_sequence_minutes=15,
    window_size_minutes=120,
    pred_size_minutes=30,
    stride=15,
)
test_loader = DataLoader(dataset, batch_size=1, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TPTrans(pred_len=dataset[0][1].shape[0]).to(device)
model.load_state_dict(torch.load("checkpoints/tptrans_delta_lin_newNewParams_175.pth", map_location=torch.device(device)))
#model.load_state_dict(torch.load("/zhome/63/7/219953/DeepLearning-1/checkpoints/tpinformer_delta_lin_newNewParams_375.pth", map_location=torch.device(device)))
model.eval()

criterion = nn.MSELoss()
test_loss = 0.0
data = []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        tgt_start = x[:, -1:, :2]  # (batch, 1, 2) lat/lon
        pred = model(x, tgt_start)
        data.append((x,y,pred))
        loss = criterion(pred, y)
        test_loss += loss.item() * x.size(0)
test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.6f}", flush=True)

plot_testresult_sample(data)

TypeError: listdir: path should be string, bytes, os.PathLike, integer or None, not SlidingWindowDataset